In [ ]:
import os
import numpy as np
import pandas as pd
import qtconsole
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import copy
try:
    import ROOT
except:
    print('Root framework not available.')

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
%qtconsole

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]
# %matplotlib notebook

# Positron emittance and yield vs. amorphous target thickness

In [ ]:
plotDefs = [
    {
        'varName1': 'x', 'varName2': 'y',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'x', 'varName2': 'px',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'y', 'varName2': 'py',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'z', 'varName2': 'pz',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'Ekin',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'pz',
        'opacityHist': 0.5,
    },
]

## Load distributions

In [ ]:
simLabels = ('17p5', '15', '12p5', '10', '7p5')
beam = {}
pars = {}

In [ ]:
sdfFilePath = '/afs/psi.ch/project/Pcubed/SimulationRuns/Geant4/000011/FCCeeTargetTracking_amor_leave_pdgId_-11.root.sdf_txt'
beam['17p5'] = bd.load_standard_fwf(sdfFilePath)
pars['17p5'] = beam['17p5'].describe()
pars['17p5']

In [ ]:
sdfFilePath = '/afs/psi.ch/project/Pcubed/SimulationRuns/Geant4/000012/FCCeeTargetTracking_amor_leave_pdgId_-11.root.sdf_txt'
beam['15'] = bd.load_standard_fwf(sdfFilePath)
pars['15'] = beam['15'].describe()
pars['15']

In [ ]:
sdfFilePath = '/afs/psi.ch/project/Pcubed/SimulationRuns/Geant4/000013/FCCeeTargetTracking_amor_leave_pdgId_-11.root.sdf_txt'
beam['12p5'] = bd.load_standard_fwf(sdfFilePath)
pars['12p5'] = beam['12p5'].describe()
pars['12p5']

In [ ]:
sdfFilePath = '/afs/psi.ch/project/Pcubed/SimulationRuns/Geant4/000014/FCCeeTargetTracking_amor_leave_pdgId_-11.root.sdf_txt'
beam['10'] = bd.load_standard_fwf(sdfFilePath)
pars['10'] = beam['10'].describe()
pars['10']

In [ ]:
sdfFilePath = '/afs/psi.ch/project/Pcubed/SimulationRuns/Geant4/000015/FCCeeTargetTracking_amor_leave_pdgId_-11.root.sdf_txt'
beam['7p5'] = bd.load_standard_fwf(sdfFilePath)
pars['7p5'] = beam['7p5'].describe()
pars['7p5']

In [ ]:
plotDefs[0]['lims1'] = (-15, 15.)   # [mm]
plotDefs[0]['lims2'] = (-15., 15.)   # [mm]
plotDefs[1]['lims1'] = plotDefs[0]['lims1']   # [mm]
plotDefs[1]['lims2'] = (-60., 60.)   # [MeV/c]
plotDefs[2]['lims1'] = plotDefs[0]['lims2']   # [mm]
plotDefs[2]['lims2'] = (-60., 60.)   # [MeV/c]
plotDefs[3]['lims1'] = (5., 20.)   # [mm]
plotDefs[3]['lims2'] = (0., 20.)   # [MeV/c]
plotDefs[4]['lims1'] = (-20e-3, 60e-3)   # [ns]
plotDefs[4]['lims2'] = plotDefs[3]['lims2']   # [MeV]
plotDefs[5]['lims1'] = plotDefs[4]['lims1']   # [ns]
plotDefs[5]['lims2'] = plotDefs[3]['lims2']   # [MeV/c]

In [ ]:
beamSel = {key: beam[key] for key in ('17p5', '12p5', '7p5')}
_ = bd.plot_distr(
    beamSel.values(), plotDefs,
    title="Positron beam at target exit",
    legendLabels=['Thickness = 17.5 mm', 'Thickness = 12.5 mm', 'Thickness = 7.5 mm']
)

<div class="alert alert-block alert-warning">
Some warning.
</div>

## Yield vs thickness

In [ ]:
totElectronsIn = 10000
thicknessArray = np.array((17.5, 15., 12.5, 10., 7.5))

In [ ]:
yieldArray = np.array([beam[lab]['Q'].shape[0] for lab in simLabels]) / totElectronsIn
sigmaXArray = np.array([pars[lab]['x']['std'] for lab in simLabels])
sigmaYArray = np.array([pars[lab]['y']['std'] for lab in simLabels])
# beamDivergenceArray = np.array([(beam[lab]['px']/beam[lab]['pz']).abs().mean() for lab in simLabels])
# xp = np.arctan(beam[lab]['px']/beam[lab]['pz'])
# xp = xp[xp]
sigmaXpArray = np.array([(np.arctan(beam[lab]['px']/beam[lab]['pz'])).std() for lab in simLabels])
sigmaYpArray = np.array([(np.arctan(beam[lab]['py']/beam[lab]['pz'])).std() for lab in simLabels])
emitnXArray = np.array([bd.compute_emittance(beam[lab], 'x') for lab in simLabels])
emitnYArray = np.array([bd.compute_emittance(beam[lab], 'y') for lab in simLabels])

In [ ]:
for lab in simLabels:
    pr = np.sqrt(beam[lab]['px']**2. + beam[lab]['py']**2.)
    phi = np.arctan(pr/beam[lab]['pz'])
    beam[lab]['phi'] = phi

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(12,16))
ax[0].plot(thicknessArray, yieldArray, 'o-')
ax[0].set_ylim([0, 15.])
ax[0].grid()
ax[0].set_ylabel('Positron yield')
ax[1].plot(thicknessArray, sigmaXArray, 'o-')
ax[1].plot(thicknessArray, sigmaYArray, 'o-')
ax[1].set_ylim([0, 1.6])
ax[1].grid()
ax[1].set_ylabel('Rms spot size [mm]')
ax[2].plot(thicknessArray, sigmaXpArray, 'o-')
ax[2].plot(thicknessArray, sigmaYpArray, 'o-')
ax[2].set_ylim([0, 0.45])
ax[2].grid()
ax[2].set_ylabel('Rms beam divergence [MeV/c]')
lPlot = ax[3].plot(thicknessArray, emitnXArray, 'o-')
ax[3].plot(thicknessArray, emitnYArray, 'o-')
ax[3].plot(
    thicknessArray, sigmaXArray*sigmaXpArray/bd.PART_CONSTS['Erest'][-11]*1e3,
    'x-', color=lPlot[0].get_color()
)
ax[3].set_ylim([0, 22e3])
ax[3].grid()
ax[3].set_ylabel('Rms norm. emittance [mm mrad]')
ax[3].set_xlabel('Amorphous target thickness [mm]')

In [ ]:
for lab in simLabels:
    _ = plt.hist(
        beam[lab]['phi'][(beam[lab]['Ekin']>0) & (beam[lab]['Ekin']<200.)],
        200, alpha=0.3
)

<div class="alert alert-block alert-danger">
Some danger.
</div><br />
<div class="alert alert-block alert-success">
Some good news.
</div>